In [22]:
from pathlib import Path
import polars as pl

from neuro.polars_utils import load_db

songs = load_db(root=Path(".."), as_db=False)

def query(field: str, text: str, df: pl.DataFrame = songs) -> pl.DataFrame:
    return df.filter(
        pl.col(field).str
        .to_lowercase().str
        .contains(text.lower())
    )
A = "Artist"
S = "Song"
q = query(A,"yes")
# q = query(S,"russ", df=q)
# q = query(S,"This i")

# songs.write_database("Songs", "sqlite:///../data/songs.db")
# songsdb = pl.read_database_uri("SELECT * FROM Songs", "sqlite://../data/songs.db")
q

id,Song,Artist,Song_ASCII,Artist_ASCII,Date,Album,Album_ID,Image,File_IN,Hash_IN,Flags,Key,Tempo (1/4 beat)
i64,str,str,str,str,str,str,i64,str,str,str,str,str,i64
463,"""Roundabout""","""Yes""","""Roundabout""","""Yes""","""2024-07-10""","""Neuro 2024-07-10 Karaoke""",6,null,"""songs/drive/Duets/Yes - Rounda…","""0f0528afed05d615487967256dc02f…","""v3;duet;neuro;""",null,null


In [14]:
tempo = pl.read_csv("../data/songs-discord.csv")
tempo_ = tempo.select([pl.col("id"), pl.col("Key"), pl.col("Tempo (1/4 beat)")])
# # Just add the columns to the songs db
# songs_ = songs.drop(["Key", "Tempo"])
# songs_ = pl.concat([songs_, tempo], how="horizontal")
# songs_ = songs.with_columns(
#     pl.lit(None).alias("Key"),
#     pl.lit(None).alias("Tempo (1/4 beat)")
# )
songs_ = songs.join(tempo_, left_on="id", right_on="id", how="left")
songs_.write_csv("../data/songs.csv")

In [1]:
from mutagen.id3 import ID3
from mutagen.id3._frames import TBPM, TKEY, TPE1, TRCK, TIT2, TALB, TDRL, TSO2, TPE2
# ID3 tags:
from pprint import pprint

# TALB = Album
# TIT2 = Title
# TRCK = Track number
# TDRL = Year-Month-Day
# TPE1 = Artist
# TPE2 = Band/Orchestra/Accompaniment
# TSO2 = Album Artist
#  TBPM = Tempo
#

k = ID3("../out/main/Mili - world.execute(me) [Neuro] [2023-06-21].mp3")

# k
for key, value in k.items():
    print(f"{key}: {value}")

TIT2: world.execute(me);
TPE1: Mili
TRCK: 2
TALB: Neuro 2023-06-21 Karaoke
TDRC: 2023
TKEY: Dm
TBPM: 130
TDRL: 2023-06-21
TPE2: Neuro-Sama
TSO2: Neuro-Sama
TENC: LAME in FL Studio 21
APIC:Cover: APIC(encoding=<Encoding.UTF8: 3>, mime='image/jpeg', type=<PictureType.ILLUSTRATION: 18>, desc='Cover', data=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x01\xf4\x01\xf4\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x1

In [25]:
from pathlib import Path
from neuro import SONGS_CSV
import polars as pl

x = pl.read_csv(Path("..") / SONGS_CSV)

cond = pl.lit(False)
cond |= pl.col("Flags").str.contains("v1")
cond |= pl.col("Flags").str.contains("v2")
cond |= pl.col("Flags").str.contains("v3")

x.with_columns(
    pl.when(cond)
    .then(pl.col("Flags"))
    .otherwise(pl.lit("v3;") + pl.col("Flags"))
    .alias("Flags")
).write_csv(Path("..") / SONGS_CSV)

# Adding new duplicates

In [9]:
from dataclasses import dataclass
from pathlib import Path
from neuro import SONGS_CSV
import polars as pl

songs = pl.read_csv(Path("..") / SONGS_CSV)

maxid = songs.select(pl.max("id")).get_column("id")[0]
L = [maxid]

@dataclass
class NewFields:
    date: str
    id: int

# Using a 1 element list to pass the int by reference
def gen_duplicate(
    df: pl.DataFrame,
    fields: NewFields,
    id: list[int],
    v1: bool = True # Change regarding which files are treated
) -> pl.DataFrame:
    if len(df) > 1:
        raise ValueError("DataFrame contains more than one song!")
    if len(df) == 0:
        raise ValueError("No songs in DataFrame!")
    row = df.row(0, named=True)

    id[0] += 1
    if v1:
        row["Album"] = f"Neuro [v1] {fields.date} Karaoke"
        row["Image"] = None
    else:
        row["Album"] = row["Album"].replace(row["Date"], fields.date)
    row["Date"] = fields.date
    row["Album_ID"] = fields.id

    row["id"] = id[0]
    if row["Flags"] is None:
        row["Flags"] = ""
    row["Flags"] += "duplicate;"
    return pl.DataFrame(row)

**Query the song and check if the new data is correct**

In [87]:
new = NewFields(
    date="2023-03-22",
    id=23
)
df = query(S, "shelter").filter(
    pl.col("Flags").str.contains("v1")
    & pl.col("Flags").str.contains("dupl").not_()
    # pl.col("Flags").is_null()
)

x = L[0]
pl.concat([
    df,
    gen_duplicate(df, new, [x])
])

id,Song,Artist,Song_ASCII,Artist_ASCII,Date,Album,Album_ID,Image,File_IN,Hash_IN,Flags,Key,Tempo (1/4 beat)
i64,str,str,str,str,str,str,i64,str,str,str,str,str,i64
880,"""Shelter""","""Porter Robinson""","""Shelter""","""Porter Robinson""","""2023-02-10""","""Neuro [v1] February Stream Son…",7,"""2023-Feb""","""songs/drive/v1 voice/[02-10-23…","""84f7a50dee249e499f175d4c65df82…","""v1;neuro;""","""C/Am""",null
1203,"""Shelter""","""Porter Robinson""","""Shelter""","""Porter Robinson""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",23,null,"""songs/drive/v1 voice/[02-10-23…","""84f7a50dee249e499f175d4c65df82…","""v1;neuro;duplicate;""","""C/Am""",null


**After reviewing, adding song to the dataframe**

In [88]:
songs = songs.extend(gen_duplicate(df, new, L))
pl.DataFrame(songs.rows(named=True)[:-10:-1])

id,Song,Artist,Song_ASCII,Artist_ASCII,Date,Album,Album_ID,Image,File_IN,Hash_IN,Flags,Key,Tempo (1/4 beat)
i64,str,str,str,str,str,str,i64,null,str,str,str,str,null
1203,"""Shelter""","""Porter Robinson""","""Shelter""","""Porter Robinson""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",23,null,"""songs/drive/v1 voice/[02-10-23…","""84f7a50dee249e499f175d4c65df82…","""v1;neuro;duplicate;""","""C/Am""",null
1202,"""Careless Whisper""","""George Micheal""","""Careless Whisper""","""George Micheal""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",25,null,"""songs/drive/v1 voice/[02-14-23…","""a58843686f7b97597fa758236bc593…","""v1;neuro;duplicate;""","""Dm""",null
1201,"""Enemy""","""Imagine Dragons""","""Enemy""","""Imagine Dragons""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",22,null,"""songs/drive/v1 voice/[02-02-23…","""fc65e5157be0f36bc06320121eec12…","""v1;neuro;duplicate;""",null,null
1200,"""After Dark""","""Mr. Kitty""","""After Dark""","""Mr. Kitty""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",20,null,"""songs/drive/v1 voice/[03-02-23…","""12d6c0b17e76d4dd00baf9dd26565c…","""v1;neuro;duplicate;""","""Gm""",null
1199,"""Pokémon Theme Song""","""Jason Paige""","""Pokemon Theme Song""","""Jason Paige""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",20,null,"""songs/drive/v1 voice/[03-15-23…","""ea6317b0d00653de8ed09132342929…","""v1;neuro;duplicate;""","""Gm""",null
1198,"""All Star""","""Smash Mouth""","""All Star""","""Smash Mouth""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",19,null,"""songs/drive/v1 voice/[03-04-23…","""b818402ed9119b6cae1d0a97e78012…","""v1;neuro;duplicate;""","""Gb""",null
1197,"""Take On Me""","""A-Ha""","""Take On Me""","""A-Ha""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",18,null,"""songs/drive/v1 voice/[02-28-23…","""321432a61522056984558c010983eb…","""v1;neuro;duplicate;""","""A""",null
1196,"""I Kissed A Girl""","""Katy Perry""","""I Kissed A Girl""","""Katy Perry""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",16,null,"""songs/drive/v1 voice/[03-07-23…","""c25a5f43be58246e3d824e3b77f02a…","""v1;neuro;duplicate;""","""Am""",null
1195,"""I Am Machine""","""Three Days Grace""","""I Am Machine""","""Three Days Grace""","""2023-03-22""","""Neuro [v1] 2023-03-22 Karaoke""",15,null,"""songs/drive/v1 voice/[02-25-23…","""34d603bc428cdac51b2e316c465a6e…","""v1;neuro;duplicate;""","""Dm""",null


**After validation of several songs, write changes to the CSV**

In [89]:
songs.write_csv(Path("..") / SONGS_CSV)